In [11]:
%load_ext autoreload
%autoreload 2

from mesa import Agent, Model
from mesa.time import StagedActivation
import numpy as np
from mesa.datacollection import DataCollector
import random
import tqdm
#from mesa.batchrunner import BatchRunner

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from mesa import Agent
from mesa.time import StagedActivation
from mesa.datacollection import DataCollector
import numpy as np
import random
np.random.seed(seed = 144)

def parents_position_in_income_distribution(agent):
    position = (agent.parent.y - agent.model.y_min) / (agent.model.y_max - agent.model.y_min)
    return position

class CorruptionAgent(Agent):
    """ Agents in the model."""
    def __init__(self, unique_id, model, parent=None):
        """Create an instance of agents with properties:
        steps = 0, counter so that they can die
        parent = parent, if it was not spawned in the beginning
        b = risk-aversion, taken from a uniform distribution, according to the paper
        k = capital endowment, initially taken from a uniform distribution,
        then inherited from parent's income and taken from distribution."""
        super().__init__(unique_id, model)
        self.steps = 0 
        self.id = unique_id
        self.parent = parent
        self.get_risk_aversion()
        self.get_capital_endowment()
        self.p=0
      
    def get_risk_aversion(self):
        self.b = np.random.uniform(self.model.b_bar-self.model.b_range/2,
                                   self.model.b_bar+self.model.b_range/2)
        
    def get_capital_endowment(self):
        if self.parent is None:
            self.k = np.random.uniform(self.model.k_bar - self.model.k_range/2, 
                self.model.k_bar + self.model.k_range/2)
        
        else:#This formula produces nan values for k, 
            self.k = (self.model.k_min + (parents_position_in_income_distribution(self)) *
                    (self.model.k_max-self.model.k_min) * (
                        1-self.model.theta) + 
                      (self.model.theta*np.random.uniform(
                          self.model.k_min, self.model.k_max)))
            if np.isnan(self.k):
                print("Agent: {}, k_min:{}, k_max: {}, y_min:{}, y_max:{}, theta: {}, y: {}".format(self.id, 
                                                                                        self.model.k_min, 
                                                                                        self.model.k_max, 
                                                                                         self.model.y_min, 
                                                                                        self.model.y_max, 
                                                                                        self.model.theta,
                                                                                        self.parent.y))           

          
    def spawn_new_agent(self):
        """Spawn new agent with the spawning agent as parent, add to model."""
        new_agent=CorruptionAgent(self.unique_id, model, parent = self)
        self.model.list_of_new_agents.append(new_agent)
        
        
             
    
    def choose_dishonesty_level(self):  
        """Choose a dishonesty level to maximize utility."""
        ###S becomes 0, why?
        if self.model.S == 0:
            self.p = 0.5
        else: 
            zwischen = (2*self.b*self.model.gamma**2*(
            1-self.model.q)*self.model.S)
            #p is larger than 1, why? S is wrong
            p = 1/zwischen
            print(f"zwischen: {zwischen}, p: {self.p}, b:{self.b}, gamma:{self.model.gamma**2}, 1-q: {1-self.model.q}, S: {self.model.S}")
        
        
    def get_income(self):
        """Get income y:
        One part is fixed, due to non-illicit corrupt work, 
        the other part due to corrupt work."""
        if not ((1-self.model.q)*self.model.gamma * self.model.S * self.k) >= 0 :
            ##Problem here: q > 1; find out why
            print(f"k:{self.k}, q:{self.model.q}, gamma:{self.model.gamma}, S:{self.model.S}")
            self.y=np.random.uniform(0,1)
        else: 
            self.y = (1-self.p)*(1-self.model.q)*self.model.S*self.k + (
            self.p * np.random.normal(self.model.S*self.k, 
         (1-self.model.q)*self.model.gamma * 
           self.model.S * self.k))
        
                
    def corrupt(self):
        ''' Choose dishonesty level, and associated income, add to steps.'''
        #check if
        self.choose_dishonesty_level()
        self.get_income()
        self.steps += 1
        
    def procreate(self):
        """ Breed new agent, then die."""
        self.spawn_new_agent()
        if self.steps >= 1:
            self.model.schedule.remove(self)
        


In [34]:
from mesa import Agent, Model
from mesa.time import StagedActivation
import numpy as np
from mesa.datacollection import DataCollector
import random


def total_capital(model):
    """ Determine total capital of the model as sum of all the agents' capitals"""
    agent_capitals = [a.k for a in model.schedule.agents]
    capital = np.sum(agent_capitals)
    return capital

def corruption_index(model):
    """Determine total corruption index of the model. 
    Weighted average of all the dishonesty levels,
    with weights being each agent's capital"""
    capital = total_capital(model)
    corruption_index = (
        np.sum([a.k*a.p for a in model.schedule.agents])) / capital
    print(capital)
    print(np.sum([a.k for a in model.schedule.agents]))
    print([a.p for a in model.schedule.agents])
    return corruption_index
    
def min_max_income(model):
    """ Determine minimum and maximum income level each generation."""
    min_income = np.min([a.y for a in model.schedule.agents])
    max_income = np.max([a.y for a in model.schedule.agents])
    return min_income, max_income

def social_capital(model):
    """Determine social capital each generation."""
    capital = total_capital(model)
    social_capital = model.alpha * capital
    return social_capital

def national_income(model):
    """Determine the national income as sum over all the individual incomes."""
    incomes = [a.y for a in model.schedule.agents]
    national_income = np.sum(incomes)
    return national_income

class CorruptionModel(Model):
    """A model with population agents."""
    def __init__(self, b_bar = 3, b_range = 1, 
                 alpha= 0.5,  gamma = 0.5, theta= 0.1, q_start= 0.1, 
                 population=1000, k_bar = 0.5, 
                 k_range = 1):
        """Create the model with the following parameters:
        Average level of risk aversion = b_bar
        Range of risk aversion = b_range
        Proportion of income spent on vigilance = gamma
        Mean human capital endowment in first generation = k_bar
        Equality in access to human capital = theta
        Initial value of social corruption = q_start
        Population = population
        Number of generations = generations
        Range of human capital endowment: k_range"""
        #Set parameters
        self.running = True
        self.num_agents= population
        self.b_bar = b_bar
        self.b_range = b_range
        self.alpha = alpha
        self.gamma = gamma
        self.theta = theta
        self.q = q_start
        self.k_bar = k_bar
        self.k_range = k_range
        self.k_min = k_bar -0.5*k_range
        self.k_max = k_bar +0.5*k_range
        self.y_min = 0
        self.S = alpha * (k_bar * population)
        self.schedule = StagedActivation(self, stage_list=["corrupt", 
                                                           "procreate"])
        self.list_of_new_agents = []
        
    
    
        #Create agents
        for i in range(self.num_agents):
            a = CorruptionAgent(i, self)
            self.schedule.add(a)       
              
        #Add data to report
        self.datacollector = DataCollector(
            model_reporters={"Total capital": total_capital, 
                             "Corruption Index": corruption_index, 
                            "National Income": national_income, 
                            "Min Income": min_max_income},
            agent_reporters={"Dishonesty": lambda a:a.p, 
                            "Income": lambda a: a.y,
                            "Capital": lambda a: a.k})
        
        
        
    def step(self):
        ''' Advance the model by one step. Do this in two stages:
        In first stage, corrupt, then report data and update model parameters.
        In second stage, breed and die.'''
        for stage in self.schedule.stage_list:
            for agent in self.schedule.agents[:]:
                getattr(agent, stage)() 
            if stage == "corrupt":
                self.q = corruption_index(self)
                self.datacollector.collect(self)
                self.y_min, self.y_max = min_max_income(self)
                print([a.y for a in self.schedule.agents])
                print([a.k for a in self.schedule.agents])
                #self.k_min, self.k_max = min_max_capital(self)
                self.K = total_capital(self)
                self.S = social_capital(self)
            for new_agent in self.list_of_new_agents:
                self.schedule.add(new_agent)
            self.list_of_new_agents = []
            
            
        self.schedule.steps += 1

    
    def run_model(self, n):
        for i in range(n):
            self.step()
    
                
    

In [43]:
model = CorruptionModel(population = 100)

model.run_model(10)

zwischen: 32.68259898885362, p: 0, b:2.9051199101203213, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 36.55897332011112, p: 0, b:3.2496865173432106, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 32.026133852593176, p: 0, b:2.846767453563838, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 28.638902954109547, p: 0, b:2.5456802625875152, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 28.412798358034706, p: 0, b:2.525582076269752, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 32.07510115247676, p: 0, b:2.851120102442379, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 38.76699242333956, p: 0, b:3.445954882074627, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 30.055880963858616, p: 0, b:2.6716338634540993, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 31.528569144662523, p: 0, b:2.8025394795255574, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 31.761784463355102, p: 0, b:2.8232697300760092, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 35.68764467746032, p: 0, b:3.1722350824409173, gamma:0.25, 1-q: 0.9, S: 25.0
zwischen: 36.824974590245176, p: 0, b:3.2

In [41]:
print(model.__dict__)

{'running': True, 'num_agents': 2, 'b_bar': 3, 'b_range': 1, 'alpha': 0.5, 'gamma': 0.5, 'theta': 0.1, 'q': 0.0, 'k_bar': 0.5, 'k_range': 1, 'k_min': 0.0, 'k_max': 1.0, 'y_min': 0.035080463740452116, 'S': 0.49311769878812606, 'schedule': <mesa.time.StagedActivation object at 0x7f5363afa9e8>, 'list_of_new_agents': [], 'datacollector': <mesa.datacollection.DataCollector object at 0x7f5363afac88>, 'y_max': 0.46197145161167652, 'K': 0.98623539757625212}


In [44]:
model.datacollector.get_model_vars_dataframe().head()

,Corruption Index,Min Income,National Income,Total capital
0,0.0,"(0.0106982383221, 22.4583272931)",1065.834292,47.370413
1,0.0,"(0.107244600115, 22.9191188418)",1136.123233,47.967630
2,0.0,"(1.56899407626, 23.1846284425)",1185.016344,49.409001
3,0.0,"(0.45689693985, 24.6509774385)",1175.667295,47.589195
4,0.0,"(0.357672642761, 21.9415892137)",1115.076940,46.862609


In [22]:
print(np.random.__file__)

/home/toni/.conda/envs/abm/lib/python3.6/site-packages/numpy/random/__init__.py


In [82]:
model.datacollector.get_agent_vars_dataframe()

Dishonesty    Income   Capital
Step AgentID                                
0    0          1.551849  0.474359  0.861777
     1          1.401062  0.242287  0.763520
1    0         -2.032799  1.000000  0.943672
     1         -1.686572  1.000000  0.012716
2    0          0.438817  1.000000       NaN
     1          0.510950  1.000000       NaN

params = {"population": 1000, 
                 "alpha" : 0.5,  
                "gamma" : 0.5, 
                "theta":  0.1, 
                "q_start" : 0.1, 
                 "k_bar" : 0.5, 
                 "k_range" : 1,
               "b_bar": [2, 3, 4],
                   "b_range" : 1}
    #"b_bar" : [3, 3.5, 4]}
batch_run = BatchRunner(CorruptionModel, 
                        parameter_values = params,
                        
                        iterations = 10, 
                        max_steps = 5, 
                        model_reporters = {
                            "Corruption Index": corruption_index, 
                            "National Income": national_income}, 
                        display_progress =True)

In [ ]:
from itertools import product
class BatchRunner:
    """ This class is instantiated with a model class, and model parameters
    associated with one or more values. It is also instantiated with model and
    agent-level reporters, dictionaries mapping a variable name to a function
    which collects some data from the model or its agents at the end of the run
    and stores it.

    Note that by default, the reporters only collect data at the *end* of the
    run. To get step by step data, simply have a reporter store the model's
    entire DataCollector object.

    """
    def __init__(self, model_cls, parameter_values, iterations=1,
                 max_steps=1000, model_reporters=None, agent_reporters=None,
                 display_progress=True):
        """ Create a new BatchRunner for a given model with the given
        parameters.

        Args:
            model_cls: The class of model to batch-run.
            parameter_values: Dictionary of parameters to their values or
                ranges of values. For example:
                    {"param_1": range(5),
                     "param_2": [1, 5, 10],
                      "const_param": 100}
            iterations: The total number of times to run the model for each
                combination of parameters.
            max_steps: The upper limit of steps above which each run will be halted
                if it hasn't halted on its own.
            model_reporters: The dictionary of variables to collect on each run at
                the end, with variable names mapped to a function to collect
                them. For example:
                    {"agent_count": lambda m: m.schedule.get_agent_count()}
            agent_reporters: Like model_reporters, but each variable is now
                collected at the level of each agent present in the model at
                the end of the run.
            display_progress: Display progresss bar with time estimation?

        """
        self.model_cls = model_cls
        self.parameter_values = {param: self.make_iterable(vals)
                                 for param, vals in parameter_values.items()}
        self.iterations = iterations
        self.max_steps = max_steps

        self.model_reporters = model_reporters
        self.agent_reporters = agent_reporters

        if self.model_reporters:
            self.model_vars = {}

        if self.agent_reporters:
            self.agent_vars = {}

        self.display_progress = display_progress

    def run_all(self):
        """ Run the model at all parameter combinations and store results. """
        params = self.parameter_values.keys()
        param_ranges = self.parameter_values.values()
        run_count = 0

        if self.display_progress:
            pbar = tqdm(total=len(list(product(*param_ranges))) * self.iterations)

        for param_values in list(product(*param_ranges)):
            kwargs = dict(zip(params, param_values))
            for _ in range(self.iterations):
                model = self.model_cls(**kwargs)
                self.run_model(model)
                # Collect and store results:
                if self.model_reporters:
                    key = tuple(list(param_values) + [run_count])
                    self.model_vars[key] = self.collect_model_vars(model)
                if self.agent_reporters:
                    agent_vars = self.collect_agent_vars(model)
                    for agent_id, reports in agent_vars.items():
                        key = tuple(list(param_values) + [run_count, agent_id])
                        self.agent_vars[key] = reports
                if self.display_progress:
                    pbar.update()
                print(run_count)

                run_count += 1

        if self.display_progress:
            pbar.close()

    def run_model(self, model):
        """ Run a model object to completion, or until reaching max steps.

        If your model runs in a non-standard way, this is the method to modify
        in your subclass.

        """
        while model.running and model.schedule.steps < self.max_steps:
            model.step()

    def collect_model_vars(self, model):
        """ Run reporters and collect model-level variables. """
        model_vars = {}
        for var, reporter in self.model_reporters.items():
            model_vars[var] = reporter(model)
        return model_vars

    def collect_agent_vars(self, model):
        """ Run reporters and collect agent-level variables. """
        agent_vars = {}
        for agent in model.schedule.agents:
            agent_record = {}
            for var, reporter in self.agent_reporters.items():
                agent_record[var] = reporter(agent)
            agent_vars[agent.unique_id] = agent_record
        return agent_vars

    def get_model_vars_dataframe(self):
        """ Generate a pandas DataFrame from the model-level variables collected.

        """
        index_col_names = list(self.parameter_values.keys())
        index_col_names.append("Run")
        records = []
        for key, val in self.model_vars.items():
            record = dict(zip(index_col_names, key))
            for k, v in val.items():
                record[k] = v
            records.append(record)
        return pd.DataFrame(records)

    def get_agent_vars_dataframe(self):
        """ Generate a pandas DataFrame from the agent-level variables
        collected.

        """
        index_col_names = list(self.parameter_values.keys())
        index_col_names += ["Run", "AgentID"]
        records = []
        for key, val in self.agent_vars.items():
            record = dict(zip(index_col_names, key))
            for k, v in val.items():
                record[k] = v
            records.append(record)
        return pd.DataFrame(records)

    @staticmethod
    def make_iterable(val):
        """ Helper method to ensure a value is a non-string iterable. """
        if hasattr(val, "__iter__") and not isinstance(val, str):
            return val
        else:
            return [val]



In [ ]:
params = {"population": 2, 
          "b_bar" :3,
          "b_range" : 1, 
          "alpha":  0.5, 
          "gamma":  0.5,
          "theta" : 0.1,
          "q_start" : 0.1, 
          "k_bar":  0.5,
          "k_range" :1}
#"b_bar" : [3, 3.5, 4]}

batch_run = BatchRunner(CorruptionModel, 
                        parameter_values = params,
                        iterations = 2, 
                        max_steps = 100, 
                        model_reporters = {
                        "Corruption Index": corruption_index, 
                        "National Income": national_income}, 
                        display_progress = False)

from IPython.core.debugger import Tracer

Tracer()()
batch_run.run_all()


In [ ]:
batch_run.get_model_vars_dataframe()


In [ ]:
print(model.__dict__)